<center>
  
# TABSYN: Tabular Data Synthesis with Diffusion Models

</center>

Two challenges regarding the extention of diffusion models to tabular data are:
1. **Diverse data types:** a single table can have different columns each containing data of different types, including numerical, categorical, text, etc.
2. **Varied distributions:** the distribution of data under different columns in a single table varry widely from column to column.

**TabSyn** addresses these challenges by introducing a latent space where tabular data of all columns are jointly represented. It then proceedes to train a diffusion model on the latent representations.
This tactic allows TabSyn to:
1. Train a single diffusion model for all data types in the dataset (i.e. Generality).
2. Optimize the distribution of latent embeddings to facilitate training of the subsequent diffusion model, thus generating higher quality synthetic data (i.e. Quality).
3. Require much fewer reverse steps during training of the diffusion model, and synthesize data faster (i.e. Speed).

In this notebook, we review and implement the TabSyn model. The notebook is organized as follows:

1. [Imports and Setup]()


2. [Default Dataset]()
    
    
3. [TabSyn Algorithm]()
    
    3.1. [Load Config]()
    
    3.2. [Make Dataset]()
    
    3.3. [Instantiate Model]()
    
    3.4. [Train Model]()
        
    3.5. [Load Pretrained Model]()
    
    3.6. [Sample Data]()
    
    3.7. [Review Synthetic Data]()


# Imports and Setup

In this section, we import all necessary libraries and modules required for setting up the environment.
Most of the libraries we need to implement TabSyn are the same as TabDDPM.
We also specify `NAME_URL_DICT_UCI`, `DATA_NAME`, `DATA_DIR` and other paths as in TabDDPM's implementation.


In [58]:
import os
import shutil
import src
import json
import pandas as pd
from pprint import pprint
import os
import numpy as np
import torch
from torch.utils.data import DataLoader

from scripts.download_dataset import download_from_uci
from scripts.process_dataset import process_data

from src.data import preprocess, TabularDataset
from src.baselines.tabsyn.pipeline import TabSyn

from src.util import visualize_default

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import os
os.getcwd()

'/fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis'

In [61]:
# os.chdir('/fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/')

In [62]:
import os
from scripts.process_dataset import process_data

In [63]:
import pandas as pd

df = pd.read_csv('/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/data/raw_data/IBM_AML_FeatureEngineered_FS1/flattened_sample.csv')
df=df.dropna()
# Ignore the The Patern ID and The Source columns
df=df.iloc[::,2::]   
# Convert the first 29 Destination and Currency columns to boolean
df.iloc[:, :29] = df.iloc[:, :29].astype(float)
df.iloc[:, 29:] = df.iloc[:, 29:].astype(float)
# Convert the last column Is Fanout to boolean
df.iloc[:, -1] = df.iloc[:, -1].astype(str)


df=df.iloc[:, ~df.columns.isin(df.columns[16:29])]  # Eclude Currency Columns for test purposes
df=df.iloc[:, ~df.columns.isin(df.columns[17:20])]  # Eclude erronous columns Columns for test purposes

/tmp/ipykernel_8002/715274848.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, :29] = df.iloc[:, :29].astype(float)
/tmp/ipykernel_8002/715274848.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 29:] = df.iloc[:, 29:].astype(float)


In [64]:
# df=df.iloc[:20,::]  # test the first 20 records
df["Is FanOut"]=np.random.choice([True, False], size=len(df))
df.iloc[:, -1] = df.iloc[:, -1].astype(str)
df.head(20)

,Destination_1,Destination_2,Destination_3,Destination_4,Destination_5,Destination_6,Destination_7,Destination_8,Destination_9,Destination_10,...,Destination_12,Destination_13,Destination_14,Destination_15,Destination_16,number_transactions_above_9k_cad,min_day_to_max_day_range,avg_transaction_frequency_in_days,transaction_frequency_variance_in_days,Is FanOut
0,6376.0,5369.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3.000000,1.66884,True
1,2118007.0,2118007.0,2118007.0,2118007.0,639683.0,449285.0,449285.0,449285.0,449285.0,22306.0,...,22306.0,22306.0,6902.0,6902.0,6902.0,13.0,5.0,0.294118,1.66884,False
2,2118007.0,2118007.0,2118007.0,449285.0,449285.0,449285.0,22306.0,22306.0,22306.0,6902.0,...,6902.0,0.0,0.0,0.0,0.0,9.0,1.0,0.083333,1.66884,True
3,927.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.66884,False
5,2915.0,1053.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.500000,1.66884,True
7,28344.0,5501.0,5343.0,5343.0,4152.0,4039.0,2644.0,881.0,881.0,881.0,...,235.0,173.0,173.0,173.0,107.0,1.0,4.0,0.222222,1.66884,True
8,5343.0,5343.0,5343.0,5343.0,881.0,881.0,235.0,235.0,235.0,235.0,...,173.0,107.0,107.0,107.0,107.0,0.0,3.0,0.187500,1.66884,True
9,5343.0,2644.0,881.0,881.0,235.0,173.0,173.0,107.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.66884,False
10,831.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.66884,False
11,19467696.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.66884,True


In [65]:
def get_column_info(df):
    column_info = {col: str(df[col].dtype).replace('float64', 'float').replace('int64', 'int').replace('object', 'str') for col in df.columns}
    return column_info

# Get column info
column_info = get_column_info(df)

# Print in desired format
print({"column_info": column_info})

{'column_info': {'Destination_1': 'float', 'Destination_2': 'float', 'Destination_3': 'float', 'Destination_4': 'float', 'Destination_5': 'float', 'Destination_6': 'float', 'Destination_7': 'float', 'Destination_8': 'float', 'Destination_9': 'float', 'Destination_10': 'float', 'Destination_11': 'float', 'Destination_12': 'float', 'Destination_13': 'float', 'Destination_14': 'float', 'Destination_15': 'float', 'Destination_16': 'float', 'number_transactions_above_9k_cad': 'float', 'min_day_to_max_day_range': 'float', 'avg_transaction_frequency_in_days': 'float', 'transaction_frequency_variance_in_days': 'float', 'Is FanOut': 'str'}}


## Prepare the Dataset Config File

In [66]:
df.columns

Index(['Destination_1', 'Destination_2', 'Destination_3', 'Destination_4',
       'Destination_5', 'Destination_6', 'Destination_7', 'Destination_8',
       'Destination_9', 'Destination_10', 'Destination_11', 'Destination_12',
       'Destination_13', 'Destination_14', 'Destination_15', 'Destination_16',
       'number_transactions_above_9k_cad', 'min_day_to_max_day_range',
       'avg_transaction_frequency_in_days',
       'transaction_frequency_variance_in_days', 'Is FanOut'],
      dtype='object')

In [67]:
import pandas as pd
import json

def generate_json_config(df):
    # Get column names
    column_names = df.columns.tolist()
    
    # Identify the target column index (last column)
    target_col_idx = [len(column_names) - 1]
    
    # Initialize lists for numerical and categorical column indices
    num_col_idx = []
    cat_col_idx = []
    
    # Initialize dictionary for column info
    column_info = {}
    
    # Determine column types and indices
    for idx, col in enumerate(df.columns):
        col_type = df[col].dtype
        
        # Fill in column_info with appropriate type
        if pd.api.types.is_integer_dtype(col_type):
            column_info[col] = "int"
            if idx != target_col_idx[0]:  # Exclude target column from num_col_idx
                num_col_idx.append(idx)
        elif pd.api.types.is_float_dtype(col_type):
            column_info[col] = "float"
            num_col_idx.append(idx)
        elif pd.api.types.is_bool_dtype(col_type):
            column_info[col] = "bool"
            cat_col_idx.append(idx)
        else:
            column_info[col] = "str"
            cat_col_idx.append(idx)
    
    # Calculate train_num and test_num
    total_rows = len(df)
    train_num = int(total_rows * 1.0)
    test_num = total_rows - train_num
    
    # Construct JSON configuration object
    config = {
        "column_names": column_names,
        "num_col_idx": num_col_idx,
        "cat_col_idx": cat_col_idx,
        "target_col_idx": target_col_idx,
        "file_type": "csv",
        "data_path": "/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/data/raw_data/IBM_AML_FeatureEngineered_FS1/train.csv",
        "test_path":"/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/data/test_data/IBM_AML_FeatureEngineered_FS1/test.csv",
        "column_info": column_info,
        "train_num": train_num,
        "test_num": test_num
    }
    
    # Print JSON object
    print(json.dumps(config, indent=4))

In [68]:
generate_json_config(df)

{
    "column_names": [
        "Destination_1",
        "Destination_2",
        "Destination_3",
        "Destination_4",
        "Destination_5",
        "Destination_6",
        "Destination_7",
        "Destination_8",
        "Destination_9",
        "Destination_10",
        "Destination_11",
        "Destination_12",
        "Destination_13",
        "Destination_14",
        "Destination_15",
        "Destination_16",
        "number_transactions_above_9k_cad",
        "min_day_to_max_day_range",
        "avg_transaction_frequency_in_days",
        "transaction_frequency_variance_in_days",
        "Is FanOut"
    ],
    "num_col_idx": [
        0,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19
    ],
    "cat_col_idx": [
        20
    ],
    "target_col_idx": [
        20
    ],
    "file_type": "csv",
    "

# AML Dataset

For more explanation of different steps in this section, please refer to TabDDPM's notebook.

In [69]:
DATA_DIR = "data/"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
SYNTH_DATA_DIR = os.path.join(DATA_DIR, "synthetic_data")
DATA_NAME = "IBM_AML_FeatureEngineered_FS1"

MODEL_PATH = "models/tabsyn"
# process data
INFO_DIR = "data_info"


In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split

def split_and_save_dataset(df, train_size=0.8, random_state=42):
    # Split the dataset
    train_df, test_df = train_test_split(df, train_size=train_size, random_state=random_state)
    
    # Save to CSV files
    train_df.to_csv('data/raw_data/IBM_AML_FeatureEngineered_FS1/train.csv', index=False)
    test_df.to_csv('data/raw_data/IBM_AML_FeatureEngineered_FS1/test.csv', index=False)
    
    # Print the shapes of the resulting datasets
    print(f"Original dataset shape: {df.shape}")
    print(f"Train dataset shape: {train_df.shape}")
    print(f"Test dataset shape: {test_df.shape}")

# Assuming you have a DataFrame called 'df'
split_and_save_dataset(df)

Original dataset shape: (90, 21)
Train dataset shape: (72, 21)
Test dataset shape: (18, 21)


In [71]:
# NAME_URL_DICT_UCI = {
#     "adult": "https://archive.ics.uci.edu/static/public/2/adult.zip",
#     "default": "https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip",
#     "magic": "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip",
#     "shoppers": "https://archive.ics.uci.edu/static/public/468/online+shoppers+purchasing+intention+dataset.zip",
#     "beijing": "https://archive.ics.uci.edu/static/public/381/beijing+pm2+5+data.zip",
#     "news": "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip",
# }

# # For shared directory you can change it to "/projects/diffusion_bootcamp/data/tabular"
# DATA_DIR = "data/"
# RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
# PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
# SYNTH_DATA_DIR = os.path.join(DATA_DIR, "synthetic_data")
# # DATA_NAME = "adult"

# MODEL_PATH = "models/tabsyn"

In [72]:
# DATA_NAME = "IBM_AML_FeatureEngineered_FS1"

In [73]:
# download data
# download_from_uci(DATA_NAME, RAW_DATA_DIR, NAME_URL_DICT_UCI)

# process data
INFO_DIR = "data_info"
process_data(DATA_NAME, INFO_DIR, DATA_DIR)

# review data
df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "train.csv"))
# visualize_default(df).head(10)

XXXXXXXXXXXXXXXXXX(72, 21)
num Col index:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], Cat col index: [20]
Processing and Saving IBM_AML_FeatureEngineered_FS1 Successfully!
Dataset Name: IBM_AML_FeatureEngineered_FS1
Total Size: 72
Train Size: 64
Test Size: 8
Number of Numerical Columns: 20
Number of Categorical Columns: 2


In [74]:
# review json file and its contents
with open(f"{PROCESSED_DATA_DIR}/{DATA_NAME}/info.json", "r") as file:
    data_info = json.load(file)
pprint(data_info)

{'cat_col_idx': [20],
 'column_info': {'0': {},
                 '1': {},
                 '10': {},
                 '11': {},
                 '12': {},
                 '13': {},
                 '14': {},
                 '15': {},
                 '16': {},
                 '17': {},
                 '18': {},
                 '19': {},
                 '2': {},
                 '20': {},
                 '3': {},
                 '4': {},
                 '5': {},
                 '6': {},
                 '7': {},
                 '8': {},
                 '9': {},
                 'categorizes': [False, True],
                 'max': 1.668839537603111,
                 'min': 1.668839537603111,
                 'type': 'categorical'},
 'column_names': ['Destination_1',
                  'Destination_2',
                  'Destination_3',
                  'Destination_4',
                  'Destination_5',
                  'Destination_6',
                  'Destination_7',
  

# TabSyn Algorithm

In this section, we will describe the design of TabSyn as well as its main hyperparameters loaded through config, which affect the model’s effectiveness. 

**TabSyn** consists of two parts:
1. A *variational auto-encoder (VAE)* which learns a joint representation space for the given tabular data.
2. A *Diffusion model* which learns the distribution of data in the joint representation space.

The figure below shows a diagram of the TabSyn model.

<p align="center">
<img src="figures/tabsyn.jpg" width="1000"/>
</p>

**VAE**

The left-side of the figure shows the VAE which operates in the original data space. The VAE itself consists of two parts: an encoder and a decoder. It also contains the corresponding tokenizer and detokenizer.
Each row of the input tabular data ($\pmb{x}$) is tokenized, then embedded by a transformer. Another transformer decodes the embeddings and a detokenizer reconstructs the table ($\pmb{\tilde{x}}$). The VAE is trained by minimizing the reconstruction loss between $\pmb{x}$ and $\pmb{\tilde{x}}$.

After the VAE is fully trained, the whole data ($\pmb{x}$) is tokenized and embedded. The embedding of each row is flattened to form a 1-dimensional vector $\pmb{z}$.
These 1-dimensional embeddings for all rows are stored on disk, and will later be used to train the diffusion model.

**Diffusion**

The right-side of the figure shows the diffusion model which operates in the latent representation space; in other words, it only *sees* the embeddings obtained by the VAE, not the original tabular data.
The diffusion model can be similarly divided into two parts: a forward process, and a reverse process.

The forward process receives the embedded data points. A single data point is denoted by $\pmb{z_0}$ in the figure. Gaussian noise is incrementally added to the embeddings in numerous incremental steps during the forward process. The number of the steps is denoted by $T$ in the figure. $T$ should be high enough that the distribution of embeddings at step $t=T$ is essentially a standard Gaussian distribution; in other words, the signal-to-noise ratio is practically zero.

The reverse process, on the other hand, learns to *predict* an earlier-step embedding (e.g. $\pmb{z_{t-\Delta t}}$) from a later-step embedding (e.g. $\pmb{z_t}$) via a neural network.

After the diffusion model is fully trained, the reverse process can estimate the data distribution at step $t=0$ if it receives a standard Gaussian distribution at step $t=T$. New data points can be synthesized by sampling from this estimated distribution.


## Load Config

In this section, we will load the configuration file that contains the hyperparameters for the TabSyn model. 

In [75]:
config_path = os.path.join("src/baselines/tabsyn/configs", f"{DATA_NAME}.toml")
raw_config = src.load_config(config_path)

pprint(raw_config)

{'loss_params': {'lambd': 0.7, 'max_beta': 0.01, 'min_beta': 1e-05},
 'model_params': {'d_token': 4, 'factor': 32, 'n_head': 1, 'num_layers': 2},
 'task_type': 'binclass',
 'train': {'diffusion': {'batch_size': 4096,
                         'num_dataset_workers': 4,
                         'num_epochs': 9},
           'optim': {'diffusion': {'factor': 0.9,
                                   'lr': 0.001,
                                   'patience': 20,
                                   'weight_decay': 0},
                     'vae': {'factor': 0.95,
                             'lr': 0.001,
                             'patience': 10,
                             'weight_decay': 0}},
           'vae': {'batch_size': 4096,
                   'num_dataset_workers': 4,
                   'num_epochs': 10}},
 'transforms': {'cat_encoding': None,
                'cat_min_frequency': None,
                'cat_nan_policy': None,
                'normalization': 'quantile',
              

The configuration file is a TOML file that contains the following hyperparameters:

1. **model_params:** specifies the structure of the transformers (both encoder and decoder) in the VAE model, including number of transformer layers, number of self-attnetion heads and token dimension.

2. **transforms:** specifies the transformations and preprocessing of the data before tokenization, such as cleaning, normalization, and encoding.
    - For preprocessing numerical features, we use the gaussian quantile transformation and replace the NaN values with mean of each row.
    - For categorical features, we use the one-hot encoding method. NaN values are left unchanged, but we have the option to replace them. We have the option to drop the values that appear with less than a given minimum frequency under each column. Furthermore, we have the option to add an extra encoding step for categorical features during tokenization.

3. **train.vae:** specifies training parameters of the VAE, including batch size, number of epochs, and number of dataset workers.

4. **train.diffusion:** specifies the same training parameters as above for the diffusion model.

5. **train.optim.vae:** specifies the parameters of the *Adam* optimizer and the `ReduceLROnPlateau` learning rate scheduler used to train the VAE. Optimizer parameters include initial learning rate and weight decay. LR scheduler parameters includer `factor` and `patience`.

6. **train.optim.diffusion:** specifies the same parameters as above for the diffusion model.

7. **loss_params:** specifies parameters of the loss function used to train the VAE including `max_beta`, `min_beta` and `lambd`.

$\beta$ is the coefficient of the KL divergence term in the VAE loss formula,

$\mathcal{L}_{vae} = \mathcal{L}_{mse} + \mathcal{L}_{ce} + \beta \mathcal{L}_{kl}$
.

Parameters `max_beta` and `min_beta` determine the range of $\beta$. $\beta$ is first set to `max_beta`. If the loss stops decreasing for a certain number of epochs (e.g. $10$ epochs), then at the end of each epoch after that (e.g. epoch $11$, $12$, etc.) $\beta$ is decreased by a factor of `lambd`,
$\beta_{new} = \lambda \beta_{curr}$,
until it reaches `beta_min`.


## Make Dataset

In this section, we pre-process the data and make a dataset object.

First, we determine transformations needed for the dataset, such as normalization and cleaning, in `transforms`. Next, using `preprocess` function we load the data from disk in arrays that contain both training and test data (`X_num` and `X_cat`), as well as the number of categories for each categorical feature (`categories`) and the number of numerical features (`d_numerical`).

We then separate the train and test data in different arrays and convert them to Pytorch tensors.
We create a dataset object (`TabularDataset`) with the train data. `TabularDataset` is a simple module which returns the tokens of a single row at a time. Each row constiutes a single data sample in TabSyn. Afterwards, we create a Dataloader for the train data using the `batch_size` and `num_workers` specified in config.

In contrast, we keep the test data as tensors (`X_test_num` and `X_test_cat`). If a GPU is available, we move these tensors to GPU so that they can be accessed by the model later on.

In [76]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# preprocess data
X_num, X_cat, categories, d_numerical = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                   transforms = raw_config["transforms"],
                                                   task_type = raw_config["task_type"])

# separate train and test data
X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

# convert to float tensor
X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# create dataset module
train_data = TabularDataset(X_train_num.float(), X_train_cat)

# move test data to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_num = X_test_num.float().to(device)
X_test_cat = X_test_cat.to(device)

# create train dataloader
train_loader = DataLoader(
    train_data,
    batch_size = raw_config["train"]["vae"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["vae"]["num_dataset_workers"],
)

No NaNs in numerical features, skipping


/fs01/home/ws_aabboud/diffusion_model_bootcamp/.venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Instantiate Model

Next, we instantiate the model using the `TabSyn` class. `TabSyn` class takes the following arguments:

1. `train_loader`: dataloader for train data.
2. `X_test_num`: numerical features of the test data.
3. `X_test_cat`: categorical features of the train data.
4. `num_numerical_features`: number of numerical features in the dataset.
5. `num_classes`: number of classes (i.e. categories) of each categorical feature in the dataset.
6. `device`: the device on which the model and data exist, either "cpu" or "cuda".

In [77]:
tabsyn = TabSyn(train_loader,
                X_test_num, X_test_cat,
                num_numerical_features = d_numerical,
                num_classes = categories,
                device = device)

`TabSyn` class has the tools to instantiate VAE and diffusion models, train both, and sample from the trained diffusion model.
We will demonstrate how to use these tools in the following sections.

## Train Model


The VAE and the diffusion model are trained independently. The following subsections explain each training process.


### A. Train VAE

First, we need to instantiate the VAE using the `instantiate_vae` method. This method takes the VAE model hyperparameters, optimizer and lr scheduler parameters from config, and instantiates them.

In [78]:
# instantiate VAE model for training
tabsyn.instantiate_vae(**raw_config["model_params"], optim_params = raw_config["train"]["optim"]["vae"])

Successfully instantiated VAE model.


Now that we have instantiated the VAE, we can train it using the `train_vae` function.
This function receives the loss hyperparameters and number of epochs from the config.
Moreover, it recieves `save_path` which is the directory where trained model checkpoints will be saved.

In [79]:
# os.makedirs(f"{MODEL_PATH}/{DATA_NAME}/vae")
# Define your paths
directory_path = f"{MODEL_PATH}/{DATA_NAME}/vae"

# Check if directory exists
if os.path.exists(directory_path):
    # Remove the existing directory and its contents
    shutil.rmtree(directory_path)

# Create the directory
os.makedirs(directory_path)

In [80]:


tabsyn.train_vae(**raw_config["loss_params"],
                 num_epochs = raw_config["train"]["vae"]["num_epochs"],
                 save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

Epoch 1/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.21it/s]


epoch: 0, beta = 0.010000, Train MSE: 18.399036, Train CE:1.262515, Train KL:1.323374, Val MSE:22.221317, Val CE:0.748228, Train ACC:0.390625, Val ACC:0.625000


Epoch 2/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.34it/s]


epoch: 1, beta = 0.010000, Train MSE: 17.956131, Train CE:1.110392, Train KL:1.260985, Val MSE:21.728600, Val CE:0.950650, Train ACC:0.437500, Val ACC:0.375000


Epoch 3/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.25it/s]


...

epoch: 7, beta = 0.010000, Train MSE: 16.679516, Train CE:0.827957, Train KL:0.999093, Val MSE:20.385313, Val CE:0.694739, Train ACC:0.476562, Val ACC:0.500000


Epoch 9/10: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.30it/s]


epoch: 8, beta = 0.010000, Train MSE: 16.228052, Train CE:0.932988, Train KL:0.977210, Val MSE:20.745817, Val CE:0.816738, Train ACC:0.367188, Val ACC:0.562500


Epoch 10/10: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.18it/s]


epoch: 9, beta = 0.010000, Train MSE: 16.241333, Train CE:0.729268, Train KL:0.959072, Val MSE:19.750341, Val CE:0.684017, Train ACC:0.531250, Val ACC:0.625000
Training time: 0.0768 mins
Successfully trained and saved the VAE model!


After training the VAE, we embed the training data with the trained encoder and store the embeddings in a direcotry specified by `vae_ckpt_dir`.

In [81]:
# embed all inputs in the latent space
tabsyn.save_vae_embeddings(X_train_num, X_train_cat,
                           vae_ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

Successfully saved pretrained embeddings on disk!


### B. Train Diffusion Model

Now that we have stored the training data embeddings, we need to load and prepare them for the diffusion model.
We load the embeddings using `load_vae_embeddings`. We normalize the embeddings by subtracting the mean and dividing by the standard deviation. Then, we create a Dataloader with the specified `batch_size` and `num_workers` from the config.

In [82]:
# load latent space embeddings
train_z, _ = tabsyn.load_latent_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))  # train_z dim: B x in_dim

# normalize embeddings
mean, std = train_z.mean(0), train_z.std(0)
train_z = (train_z - mean) / std
latent_train_data = train_z

# create data loader
latent_train_loader = DataLoader(
    latent_train_data,
    batch_size = raw_config["train"]["diffusion"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["diffusion"]["num_dataset_workers"],
)

Now that the data is ready, we instantiate the diffusion model with `instantiate_diffusion`. The input dimension and hidden dimention of the diffusion model is determined by the dimension of the embeddings. 
Moreover, we instantiate the optimizer and lr scheduler using hyperparameters from config.

In [83]:
# instantiate diffusion model for training
tabsyn.instantiate_diffusion(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = raw_config["train"]["optim"]["diffusion"])

MLPDiffusion(
  (proj): Linear(in_features=88, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=88, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10674264
Successfully instantiated diffusion model.


We train the diffusion model with `train_diffusion` function.
This function takes the following arguements:
1. `latent_train_loader`: dataloader for the latent representations which are used to train the diffusion model.
2. `num_epochs`: number of training epochs.
3. `ckpt_path`: directory where the model checkpoints will be stored.

In [84]:
# os.makedirs(f"{MODEL_PATH}/{DATA_NAME}")
# train diffusion model
tabsyn.train_diffusion(latent_train_loader,
                       num_epochs = raw_config["train"]["diffusion"]["num_epochs"],
                       ckpt_path = os.path.join(MODEL_PATH, DATA_NAME))

Epoch 9/9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.55it/s, Loss=1.62]

Epoch 9/9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.51it/s, Loss=1.62]

Time:  4.230231761932373


## Load Pretrained Model

Instead of training model from scratch, we can also load weights of a pre-trained model from a given checkpoint with `load_model_state` function.
If we haven't instantiated the VAE and diffusion model beforehand, we need to instantiate them first using `instantiate_vae` and `instantiate_diffusion` methods.

In [87]:
# instantiate VAE model
tabsyn.instantiate_vae(**raw_config["model_params"], optim_params = None)


Successfully instantiated VAE model.


In [92]:
os.getcwd()

'/fs01/home/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis'

In [94]:
# latent_embeddings_path = "/projects/diffusion_bootcamp/models/tabular/tabsyn/default/vae"
latent_embeddings_path = f"/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/models/tabsyn/{DATA_NAME}/vae"  #f"/models/tabsyn/{DATA_NAME}/vae"
# load latent embeddings of input data
train_z, token_dim = tabsyn.load_latent_embeddings(latent_embeddings_path)

In [95]:
# instantiate diffusion model
tabsyn.instantiate_diffusion(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = None)

MLPDiffusion(
  (proj): Linear(in_features=88, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=88, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10674264
Successfully instantiated diffusion model.


In [96]:

pretrained_model_path = f"/h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/models/tabsyn/{DATA_NAME}"#"/projects/diffusion_bootcamp/models/tabular/tabsyn/default"
# load state from checkpoint
tabsyn.load_model_state(ckpt_dir = pretrained_model_path,
                        dif_ckpt_name = "model.pt")

Loaded model state from /h/ws_aabboud/diffusion_model_bootcamp/deloitte_team/single_table_synthesis/models/tabsyn/IBM_AML_FeatureEngineered_FS1


## Sample Data

Now that we trained the model effectively, using `sample` function we can generate synthetic data starting from compelete noise. The input of this function is as follows:

1. `train_z`: latent embeddings of the training data.
2. `info`: info about the data from the json file we reviewed at the beginning of this notebook.
3. `num_inverse`: detokenizer for numerical features.
4. `cat_inverse`: detokenizer for categorical features.
5. `save_path`: file-path where the synthetic table will be saved.

In [98]:
# load data info file
with open(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "info.json"), "r") as file:
    data_info = json.load(file)
data_info["token_dim"] = token_dim

# get inverse tokenizers
_, _, categories, d_numerical, num_inverse, cat_inverse = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                                     transforms = raw_config["transforms"],
                                                                     task_type = raw_config["task_type"],
                                                                     inverse = True)



No NaNs in numerical features, skipping


In [102]:
# sample data
num_samples = train_z.shape[0]
in_dim = train_z.shape[1] 
mean_input_emb = train_z.mean(0)
tabsyn.sample(num_samples,
              in_dim,
              mean_input_emb,
              info = data_info,
              num_inverse = num_inverse,
              cat_inverse = cat_inverse,
              save_path = os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

(10, 2)


KeyError: 21

In [106]:
train_z.shape[0],train_z.shape[1] 

(64, 88)

## Review Synthetic Data

Finally here, we review the synthesized data. In the following `evaluate_synthetic_data.ipynb` notebook, we will evaluate this synthesized data with respect to various metrics.

In [99]:
df = pd.read_csv(os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

# Display the first few rows of the DataFrame
visualize_default(df).head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'data/synthetic_data/IBM_AML_FeatureEngineered_FS1/tabsyn.csv'

## References

**Zhang, Hengrui, et al.** "Mixed-type tabular data synthesis with score-based diffusion in latent space." *International Conference on Learning Representations (ICLR)* (2023).

**GitHub Repository:** [Amazon Science - Tabsyn](https://github.com/amazon-science/tabsyn)